In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("DS1_C6_S6_Sales_Data.csv")


In [3]:
pd.set_option("display.max_columns", None)
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,24-02-2003,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,07-05-2003,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,01-07-2003,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,25-08-2003,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10-10-2003,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,NaN,San Francisco,CA,28034,USA,NaN,Brown,Julie,Medium


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [5]:
# DATA CLEANING

# ORDERDATE TO DATETIME FORMAT

df["ORDERDATE"] = pd.to_datetime(df["ORDERDATE"], format="%d-%m-%Y", errors="coerce")

# MISSING VALUES

df["STATE"] = df["STATE"].fillna("Unknown")
df["TERRITORY"] = df["TERRITORY"].fillna("Unknown")
df["ADDRESSLINE2"] = df["ADDRESSLINE2"].fillna("Unknown")

# COUNTRY VALUES

df["COUNTRY"] = df["COUNTRY"].replace({
    "USA": "United States",
    "us": "United States",
    "UK": "United Kingdom",
    "U.K.": "United Kingdom"
})

# Fix TERRITORY for USA and Canada
df.loc[df['COUNTRY'].isin(['United States', 'Canada']), 'TERRITORY'] = 'AMER'
df.loc[df['COUNTRY'].isin(['Japan', 'Singapore', 'Philippines']), 'TERRITORY'] = 'APAC'

# Validate SALES column
df["CALC_SALES"] = df["QUANTITYORDERED"] * df["PRICEEACH"]
df.loc[df["SALES"] != df["CALC_SALES"], "SALES"] = df["CALC_SALES"]
df = df.drop(columns=["CALC_SALES"])

# STATE COLUMN

city_to_state = {
    "Aaarhus": "Aarhus",
    "Barcelona": "Catalonia",
    "Bergamo": "Lombardy",
    "Bergen": "Vestland",
    "Boras": "Västra Götaland",
    "Bruxelles": "Brussels-Capital Region",
    "Charleroi": "Wallonia",
    "Dublin": "Leinster",
    "Espoo": "Uusimaa",
    "Frankfurt": "Hesse",
    "Gensve": "Geneva",
    "Graz": "Styria",
    "Helsinki": "Uusimaa",
    "Kobenhavn": "Capital Region",
    "Koln": "North Rhine-Westphalia",
    "Lille": "Hauts-de-France",
    "Liverpool": "Merseyside",
    "London": "Greater London",
    "Lule": "Norrbotten",
    "Lyon": "Auvergne-Rhône-Alpes",
    "Madrid": "Community of Madrid",
    "Makati City": "Metro Manila",
    "Manchester": "Greater Manchester",
    "Marseille": "Provence-Alpes-Côte d'Azur",
    "Munich": "Bavaria",
    "Nantes": "Pays de la Loire",
    "Oslo": "Oslo County",
    "Oulu": "Northern Ostrobothnia",
    "Paris": "Île-de-France",
    "Reggio Emilia": "Emilia-Romagna",
    "Reims": "Grand Est",
    "Salzburg": "Salzburg",
    "Sevilla": "Andalusia",
    "Singapore": "Singapore",
    "Stavern": "Vestfold og Telemark",
    "Strasbourg": "Grand Est",
    "Torino": "Piedmont",
    "Toulouse": "Occitanie",
    "Versailles": "Île-de-France"
}

for city, state in city_to_state.items():
    df.loc[(df['STATE'] == "Unknown") & (df['CITY'] == city), 'STATE'] = state



In [7]:
# EXPORT CLEANED DATA
df.to_csv("Sales_data.csv", index=False)
